Dorothée Morand-Grondin

Working on Ingrid Demoly research project - Physiological analysis

First script: cleaning data, looping over all participants

In [ ]:
# import libraries
import os
import pandas as pd
import matplotlib.pyplot as plt
import neurokit2 as nk
import numpy as np
from scipy.interpolate import interp1d

# subject IDs
subjects = [f"Subject{str(i).zfill(2)}" for i in range(1,83)]

# helper function to identify ectopic beats (useful for second round of cleaning)
def identify_ectopic(time_diffs):
    
    ectopic_beats = [] # create a list to store them

    for RR in range(len(time_diffs)):
        prev_rr = time_diffs[RR - 1] if RR > 0 else np.nan # get the previous RR
        next_rr = time_diffs[RR + 1] if RR < len(time_diffs) - 1 else np.nan # get the next RR
        current_rr = time_diffs[RR] # get the current RR

        # skip if current RR is NaN (no RR interval to test)
        if np.isnan(current_rr):
            continue

        # find valid neighbors
        valid_neighbors = [x for x in [prev_rr, next_rr] if not np.isnan(x)]

        if len(valid_neighbors) == 0:
            # if no neighbors, flag automatically
            ectopic_beats.append(RR)
            continue

        mean_neighbor = np.mean(valid_neighbors) # mean of adjacent neighbors

        # flag as ectopic if >20% than the adjacent mean
        if abs(current_rr - mean_neighbor) > 0.2 * mean_neighbor:
            ectopic_beats.append(RR)

    return ectopic_beats



In [ ]:


for subject in subjects: 
    
    print(f'processing {subject}') # tracking
    
    # extract data file
    ecg_file = f"data/{subject}_ECG.csv" 

    # formality: force consistent dtypes and rename column
    df_ecg = pd.read_csv(ecg_file, dtype={"Marqueurs": str}) # read as a string column
    df_ecg.rename(columns={"Time": "time"}, inplace=True) # time column
    df_ecg.rename(columns={"EKG": "amplitude"}, inplace=True) # signal column
    df_ecg.rename(columns={"Marqueurs": "markers"}, inplace=True) # marker column

    ECG_signal = df_ecg["amplitude"].values # identify the signal column

    # compute sampling rate from time column
    dt = np.diff(df_ecg["time"].values[:100]).mean()
    sampling_rate = int(1 / dt) # sampling rate

    # clean ECG with NeuroKit
    ecg_cleaned = nk.ecg_clean(ECG_signal, sampling_rate=sampling_rate)

    # process ECG data with NeuroKit
    signals, info = nk.ecg_process(ecg_cleaned, sampling_rate=sampling_rate)

    # plot of original detection (saved file)
    nk.ecg_plot(signals, info)
    plt.gcf().set_size_inches(18, 10)   # make it bigger before saving
    plt.savefig(f"plots/{subject}_ECG.png", dpi=300, bbox_inches="tight") # save plot
    
    print('   data extracted') # tracking
    
    ##################################################################################################################################
    # CLEANING 1: 90% quality cut-off

    rpeaks = info["ECG_R_Peaks"] # list of all peak times

    peak_quality = signals["ECG_Quality"].iloc[rpeaks].values # look at the quality at the time of each peak

    df_rpeaks = pd.DataFrame({
        "peak_time": rpeaks / sampling_rate,
        "quality": peak_quality})

    df_rpeaks["is_valid"] = df_rpeaks["quality"] >= 0.9 # flag validity < 90% in a new column ** CHANGE HERE IF NEEDED
    
    # make a new column including only the valid points
    df_rpeaks["first_clean_rpeaks"] = df_rpeaks["peak_time"].where(df_rpeaks["is_valid"], np.nan)
    
    print(f'   first cleaning done!')
    
    # add heart rate column
    # compute RR intervals (in seconds) from cleaned peaks
    rr_diff = np.diff(df_rpeaks["first_clean_rpeaks"])   # puts NaN if invalid neighbor
    hr_bpm = 60 / rr_diff

    # align to peaks (diff shortens by 1 so shift by 1 row)
    df_rpeaks.loc[1:, "time_diffs"] = rr_diff
    df_rpeaks.loc[1:, "heart_rate"] = hr_bpm
    
    ######## make a graph
    
    fig, ax = plt.subplots(figsize=(15,4)) # size of plot
    ax.plot(df_rpeaks["peak_time"], df_rpeaks["heart_rate"], "o-", # plot HR (break line at invalid areas)
            markersize=3, color="red", alpha=0.7, label="Heart Rate (1st cleaning)")

    for i, is_valid in enumerate(df_rpeaks["is_valid"]): # grey shade for invalid regions
        if not is_valid:
            t = df_rpeaks["peak_time"].iloc[i]
            ax.axvspan(t-0.1, t+0.1, color="lightgrey")
            
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Heart Rate (bpm)")
    ax.set_title(f"HR after first cleaning "
                f"({(~df_rpeaks['is_valid']).mean()*100:.1f}% removed)")
    ax.legend()

    plt.tight_layout() 
    plt.savefig(f"figures_de_suivi/{subject}_HR_first_cleaning.png", dpi=300, bbox_inches="tight") # saving in folder
    plt.close()

    ##################################################################################################################################
    # CLEANING 2: ectopic beat detection
    
    # find ectopics in the current data
    target_ectopic = identify_ectopic(df_rpeaks["time_diffs"].values)

    # force-drop first and last RR interval
    if len(df_rpeaks) > 2:  
        target_ectopic = list(set(target_ectopic) | {0, len(df_rpeaks)-1})
    
    # add ectopic flag column
    df_rpeaks["is_ectopic"] = False
    df_rpeaks.loc[target_ectopic, "is_ectopic"] = True

    # create second-clean column
    df_rpeaks["second_clean_rpeaks"] = df_rpeaks["first_clean_rpeaks"].copy()
    df_rpeaks.loc[df_rpeaks["is_ectopic"], "second_clean_rpeaks"] = np.nan

    # recompute RR intervals + HR after ectopic removal
    rr_diff2 = np.diff(df_rpeaks["second_clean_rpeaks"])
    hr_bpm2 = 60 / rr_diff2
    df_rpeaks["time_diffs_2"] = np.nan
    df_rpeaks["heart_rate_2"] = np.nan
    df_rpeaks.loc[1:, "rr_diff_2"] = rr_diff2
    df_rpeaks.loc[1:, "heart_rate_2"] = hr_bpm2

    print(f"   second cleaning done! ({len(target_ectopic)-2} ectopics removed)") # tracking
    
    ####### make a plot
    
    fig, ax = plt.subplots(figsize=(15,4))
    ax.plot(df_rpeaks["peak_time"], df_rpeaks["heart_rate_2"], "o-",
            markersize=3, color="red", alpha=0.8, label="2nd cleaning")

    # shade cleaning 1
    for i, is_valid in enumerate(df_rpeaks["is_valid"]):
        if not is_valid:
            t = df_rpeaks["peak_time"].iloc[i]
            ax.axvspan(t-0.1, t+0.1, color="lightgrey",
                    label="low quality" if i==0 else "")

    # shade cleaning 2
    for i, is_ectopic in enumerate(df_rpeaks["is_ectopic"]):
        if is_ectopic:
            t = df_rpeaks["peak_time"].iloc[i]
            ax.axvspan(t-0.1, t+0.1, color="orange",
                    label="ectopic" if i==0 else "")
            
    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Heart Rate (bpm)")
    ax.set_title("Heart Rate after 1st vs 2nd cleaning")
    ax.legend()

    plt.tight_layout()
    plt.savefig(f"figures_de_suivi/{subject}_HR_second_cleaning.png", dpi=300, bbox_inches="tight") # saving in folder
    plt.close()

    ####################################################################################################################################
    # INTERPOLATION
    
    df_rr = pd.DataFrame({ #  new df cleaned
        "rr_peak_time": df_rpeaks["peak_time"],
        "rr_interval": df_rpeaks["rr_diff_2"],
        "is_valid": ~df_rpeaks["rr_diff_2"].isna()})

    mask = df_rr["is_valid"]
    f = interp1d(
        df_rr["rr_peak_time"][mask],
        df_rr["rr_interval"][mask],
        kind="linear", # could be "linear", "quadratic", "cubic"
        fill_value="extrapolate")

    # interpolated intervals
    df_rr["rr_interval_interpol"] = f(df_rr["rr_peak_time"])
    df_rr["hr_interp_bpm"] = 60 / df_rr["rr_interval_interpol"]

    # start with a copy of original times
    rpeaks_final = df_rr["rr_peak_time"].copy().values

    # re-compute invalid peak times cumulatively using interpolated intervals
    for i in range(1, len(rpeaks_final)):
        if not df_rr["is_valid"].iloc[i]:  # if invalid, reconstruct from previous + interpolated RR
            rpeaks_final[i] = rpeaks_final[i-1] + df_rr["rr_interval_interpol"].iloc[i]

    df_rr["rr_time_interpol"] = rpeaks_final

    df_compact = df_rr[[
        "rr_peak_time",
        "rr_interval",
        "is_valid",
        "rr_interval_interpol",
        "rr_time_interpol"
    ]].rename(columns={
        "rr_peak_time": "rr_time_original",
        "rr_interval": "rr_interval_original",
        "rr_interval_interpol": "rr_interval_interpol",
        "rr_time_interpol": "rr_time_interpol"})

    out_csv = f"cleaned_data/{subject}_cleaned_ECG.csv" # saving cleaned data
    df_compact.to_csv(out_csv, index=False)
    print(f"   saved interpolated data") # tracking
    
    ####################################################################################################################################
    # FINAL PLOT

    fig, ax = plt.subplots(figsize=(15,4))

    # raw valid HR points
    ax.plot(df_rr["rr_peak_time"][df_rr["is_valid"]],
            60 / df_rr["rr_interval"][df_rr["is_valid"]],
            "o", markersize=2, color="red", alpha=0.7, label="Valid HR")

    # interpolated continuous HR
    ax.plot(df_rr["rr_peak_time"], df_rr["hr_interp_bpm"],
            "-", color="green", linewidth=1.2, label="Interpolated HR")

    # shade invalids
    for i, is_bad in enumerate(~df_rr["is_valid"]):
        if is_bad:
            t = df_rr["rr_peak_time"].iloc[i]
            ax.axvspan(t-0.5, t+0.5, color="lightgrey", alpha=0.4)

    # filter and label 'debut' markers (stimuli)
    df_debut_markers = df_ecg[df_ecg["markers"].str.startswith("Debut", na=False)].copy()

    # extract emotion name (e.g., "Debut_joie" becomes "joie")
    df_debut_markers["emotion"] = df_debut_markers["markers"].str.replace("Debut_", "", regex=False)

    # assign emotion-specific counters
    emotion_counts = {}
    labels = []

    for _, row in df_debut_markers.iterrows():
        emotion = row["emotion"]
        emotion_counts[emotion] = emotion_counts.get(emotion, 0) + 1
        label = f"{emotion}_{emotion_counts[emotion]}"
        labels.append(label)

    df_debut_markers["label"] = labels

    # plot marker lines and labels
    for _, row in df_debut_markers.iterrows():
        x = row["time"]
        label = row["label"]

        # vertical line
        ax.axvline(x=x, color="red", linestyle="--", alpha=0.8, linewidth=0.5)

        # text label at the top
        ax.text(x, 
                ax.get_ylim()[1] * 0.98, # near top 
                label,
                color="red",
                fontsize=6,
                rotation=90,
                ha = "center",
                va = "top",
                alpha=0.8)

    ax.set_xlabel("Time (s)")
    ax.set_ylabel("Heart Rate (bpm)")
    ax.set_title(f"HR with ectopic/invalid beats interpolated "
                f"({(~df_rr['is_valid']).mean()*100:.1f}% interpolated)")
    ax.legend()
    
    plt.tight_layout()
    out_png = os.path.join(f"final_figures/{subject}_HR_final.png")
    plt.savefig(out_png, dpi=300, bbox_inches="tight")
    plt.close()

    print(f"   saved final plot") # tracking

